Проект SQL - Анализ базы книг

## Описание

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

- [Полное описание задачи (Notion)](https://concrete-web-bad.notion.site/SQL-d5462ca554014f1b9be3e6f78854750f)


## Решения

- [№1 Посчитайте, сколько книг вышло после 1 января 2000 года;](#task_1)
- [№2 Для каждой книги посчитайте количество обзоров и среднюю оценку;](#task_2)
- [№3 Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;](#task_3)
- [№4 Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;](#task_4)
- [№5 Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.](#task_5)


- [Общие выводы](#itog)

### Settings

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [2]:
# упростим select
def select(sql):
    return pd.io.sql.read_sql(sql, con = engine)

## Анализ БД
Смотрим таблицы 
- `books`
- `authors`
- `publishers`
- `ratings`
- `reviews`

и анализируем структуру

In [3]:
# table → books
q_books = '''
            SELECT * FROM books
          '''
select(q_books).head()


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# table → authors
q_authors = '''
                SELECT * FROM authors
            '''
select(q_authors).head() 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# table → publishers
q_pub = '''
            SELECT * FROM publishers
        '''
publishers = select(q_pub)
publishers.head() 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
# table → ratings
q_ratings = '''
                SELECT * FROM ratings
            '''
select(q_ratings).head() 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
# table → reviews
q_reviews = '''
                SELECT * FROM reviews
            '''
select(q_reviews).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задача №1 - Посчитайте количество книг, выпущенных после 1 января 2000 года <a class="anchor" id="task_1"></a>

In [8]:
q_task1 = '''
            SELECT 
                COUNT(book_id) AS "Кол-во книг после 1/01/2000"
            FROM 
                books 
            WHERE 
                publication_date > '2000-01-01'
          '''
select(q_task1)

,Кол-во книг после 1/01/2000
0,819


### Задача №1 - Решение
После 1 января 2000 г. вышло `819 книг`

## Задача №2 - Для каждой книги посчитайте количество обзоров и среднюю оценку <a class="anchor" id="task_2"></a>

In [9]:
q_task2 = '''                              
            SELECT 
                books.title AS "Название книги",
                -- books.book_id AS "ID",
                ROUND(AVG(ratings.rating), 1) AS "Cред.оценка",
                COUNT(DISTINCT reviews.text) AS "Кол-во рецензий"
            FROM
                books
            INNER JOIN 
                ratings ON ratings.book_id = books.book_id
            LEFT JOIN 
                reviews ON reviews.book_id = books.book_id
            GROUP BY
                 books.title,
                 books.book_id
            ORDER BY
                 "Кол-во рецензий" DESC
            
        ''' 
select(q_task2)

,Название книги,Cред.оценка,Кол-во рецензий
0,Twilight (Twilight #1),3.7,7
1,Water for Elephants,4.0,6
2,The Glass Castle,4.2,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,4.4,6
4,The Curious Incident of the Dog in the Night-Time,4.1,6
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,3.7,0
996,The Natural Way to Draw,3.0,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,5.0,0
998,Essential Tales and Poems,4.0,0


### Задача №2 - Решение

- У книги `Twilight (Twilight #1)` больше всего обзоров - `7`, средняя оценка - `3.7`
- 15 книг имеют более 5 обзоров

## Задача №3 - Определите издательство, которое выпустило наибольшее число книг толще 50 страниц <a class="anchor" id="task_3"></a>
— так вы исключите из анализа брошюры;

In [10]:
# join'им две таблицы по полю publisher_id
q_task3 = '''
            SELECT 
                -- publishers.publisher_id AS id, 
                publishers.publisher AS "Издатель", 
                COUNT(books.book_id) AS "Кол-во книг"
            FROM 
                publishers
            JOIN 
                books 
            ON 
                publishers.publisher_id = books.publisher_id
            WHERE 
                books.num_pages > 50
            GROUP BY 
                publishers.publisher_id
            ORDER BY 
                "Кол-во книг" DESC
            LIMIT 10
         ''' 
select(q_task3)

,Издатель,Кол-во книг
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,Delta,13


### Задача №3 - Решение
Больше всего книг выпустило издательство `Penguin Books` - `42 книги`

## Задача №4 - Определите автора с самой высокой средней оценкой книг <a class="anchor" id="task_4"></a>
— учитывайте только книги с 50 и более оценками;

In [11]:
q_task4 = '''
            SELECT
                authors.author_id AS id,
                authors.author AS author,
                ROUND (AVG(i.avg_rating),1) AS avg_rating,
                ROUND (sum(i.count_rating),1) AS count_rating
            FROM
               (
                SELECT 
                    books.book_id, 
                    books.author_id,
                    AVG (ratings.rating) AS avg_rating,
                    COUNT (ratings.rating) AS count_rating
                FROM 
                    books
                INNER JOIN 
                    ratings ON ratings.book_id = books.book_id
                GROUP BY books.book_id,  books.author_id
                HAVING COUNT(ratings.rating) >= 50 
                ) 
                as i
                
            INNER JOIN
                authors on i.author_id = authors.author_id
            GROUP BY authors.author_id
            ORDER BY avg_rating DESC
            LIMIT 1
                        
          '''
select(q_task4)

,id,author,avg_rating,count_rating
0,236,J.K. Rowling/Mary GrandPré,4.3,310.0


### Задача №4 - Решение
Автор с самой высокой средней оценкой книг (50 и более) - `J.K. Rowling/Mary GrandPré`

## Задача №5 - Посчитайте среднее количество обзоров от пользователей <a class="anchor" id="task_5"></a>
- которые поставили больше 48 оценок.

In [12]:
# получаем usernames с 48 оценками и более
# потом из них считаем кол-во обзоров
# и уже потом средний рейтинг

q_task5 = '''
            SELECT
                ROUND(AVG(review.count_review)) AS "Среднее кол-во обзоров"
            FROM
                (
                    SELECT 
                        COUNT(review_id) AS count_review
                    FROM 
                        reviews
                    WHERE
                        username
                    IN 
                        (
                            SELECT 
                                username
                            FROM
                                ratings
                            GROUP BY
                                username
                            HAVING
                                COUNT(ratings) > 48
                        )
                    GROUP BY 
                        username
                ) 
            AS
                review;
         '''
select(q_task5)

,Среднее кол-во обзоров
0,24.0


### Задача №5 - Решение
Пользователи оставившие более `48 оценок`, написали `в среднем 24 обзора`

## Общие выводы <a class="anchor" id="itog"></a>
- После 1 января 2000 г. вышло `819 книг`
- Больше всего обзоров у книги `Twilight (Twilight #1)` - `7 обзоров`
- 15 книг имеют более 5 обзоров
- Больше всего книг выпустило издательство `Penguin Books` - `42 книги`
- Автор с самой высокой средней оценкой книг (50 и более) - `J.K. Rowling/Mary GrandPré`
- Пользователи оставившие более `48 оценок`, написали `в среднем 24 обзора`
